In [1]:
from ipykernel.comm import Comm
from IPython.display import HTML
from IPython import get_ipython
import time

In [57]:
# `target` is just a unique string key. We could generate a new one as a UUID for each `Visualizer`
target = "my_comm_target_name"


# The body of the iframe. This contains a placeholder `div` (which could become a full MeshCat visualizer)
# and a little bit of javascript to register a comm channel using the name `target`
srcdoc = """
<html>
<body>

<div id="visualizer">
(no data received yet)
</div>

</body>

<script>
// Get a handle to the `visualizer` div
const visualizer = document.getElementById("visualizer");

// Set the contents of `visualizer`. Eventually, this could easily call the actual 
// command handler in meshcat.
function handle_message(msg) {
  console.log("handle_message:", msg);
    visualizer.innerHTML = msg["command"];
}

// Register a communication channel, and call `handle_message` on each new command update
window.parent.Jupyter.notebook.kernel.comm_manager.register_target('%s',
    function(comm, msg) {
        comm.on_msg(function(msg) {
          console.log('msg:', msg);
          handle_message(msg["content"]["data"]);
        });
        comm.on_close(function(msg) {console.log('closed')});
    });


</script>
</html>
""" %target

# It's not web dev if you're not escaping something!
def srcdoc_escape(text):
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("\"", "&quot;")


# Render an iframe using the above `srcdoc` as its body. 
HTML("""
<div>
<iframe srcdoc="{:s}"></iframe>
</div>
""".format(srcdoc_escape(srcdoc)))


In [59]:
# Open the channel from Python.
command_channel = Comm(target_name=target)

In [60]:
# Verify that we can send messages and cause things to happen in javascript
for i in range(10):
    command_channel.send({"command": i})
    time.sleep(1)